# Importing Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sb

In [2]:
df=pd.read_csv('Original Data/customer_churn_dataset-training-master.csv')

In [3]:
df.sample(5)

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
18227,18236.0,59.0,Male,56.0,26.0,10.0,30.0,Premium,Monthly,220.00,13.0,1.0
51001,51837.0,57.0,Female,7.0,17.0,3.0,0.0,Premium,Annual,835.00,28.0,1.0
337852,344967.0,28.0,Male,34.0,16.0,0.0,13.0,Standard,Annual,846.75,24.0,0.0
259723,266837.0,36.0,Female,46.0,21.0,0.0,14.0,Premium,Annual,511.74,13.0,0.0
282792,289906.0,42.0,Female,2.0,2.0,4.0,19.0,Premium,Annual,768.49,11.0,0.0


In [4]:

df_dict=pd.read_excel('Original Data/Dictionary.xlsx')


In [5]:
df_dict.set_index('Column',inplace=True)

In [6]:
df_dict

,Description
Column,
CustomerID,A unique identifier for each customer
Age,The age of the customer
Gender,Gender of the customer
Tenure,Duration in months for which a customer has be...
Usage Frequency,Number of times that the customer has used the...
Support Calls,Number of calls that the customer has made to ...
Payment Delay,Number of days that the customer has delayed t...
Subscription Type,Type of subscription the customer has choosen
Contract Length,Duration of the contract that the customer has...


In [7]:
df_dict.loc['Age']

Description     The age of the customer
Name: Age, dtype: object

# Preprcossing

In [8]:
print('Number Of Rows :',df.shape[0])
print('Number Of Columns :',df.shape[1])

Number Of Rows : 440833
Number Of Columns : 12


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440833 entries, 0 to 440832
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   CustomerID         440832 non-null  float64
 1   Age                440832 non-null  float64
 2   Gender             440832 non-null  object 
 3   Tenure             440832 non-null  float64
 4   Usage Frequency    440832 non-null  float64
 5   Support Calls      440832 non-null  float64
 6   Payment Delay      440832 non-null  float64
 7   Subscription Type  440832 non-null  object 
 8   Contract Length    440832 non-null  object 
 9   Total Spend        440832 non-null  float64
 10  Last Interaction   440832 non-null  float64
 11  Churn              440832 non-null  float64
dtypes: float64(9), object(3)
memory usage: 40.4+ MB


In [10]:
df[df.isnull().all(axis=1)]

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
199295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df=df.dropna(how='all')

In [12]:
str_cols=['Gender','Subscription Type','Contract Length']
for col in str_cols:
    df[col] = df[col].str.strip()

In [13]:
num_cols = ['CustomerID','Age', 'Tenure', 'Usage Frequency', 'Support Calls', 'Payment Delay', 'Last Interaction', 'Churn']
for col in num_cols:
    df[col]=df[col].astype(int)

In [14]:
df.isnull().sum()

CustomerID           0
Age                  0
Gender               0
Tenure               0
Usage Frequency      0
Support Calls        0
Payment Delay        0
Subscription Type    0
Contract Length      0
Total Spend          0
Last Interaction     0
Churn                0
dtype: int64

In [15]:
df.duplicated().sum()

np.int64(0)

In [16]:
df['CustomerID'].duplicated().sum()

np.int64(0)

In [17]:
numeric_cols = ['Age', 'Tenure', 'Usage Frequency', 'Support Calls', 'Payment Delay', 'Last Interaction', 'Churn','Total Spend']

for col in numeric_cols:
  Q1 = df[col].quantile(0.25)
  Q3 = df[col].quantile(0.75)
  IQR = Q3 - Q1
  lower = Q1 - 1.5 * IQR
  upper = Q3 + 1.5 * IQR

  outliers = df[(df[col] < lower) | (df[col] > upper)][col]
  print(f"{col}: {len(outliers)} outliers")



Age: 0 outliers
Tenure: 0 outliers
Usage Frequency: 0 outliers
Support Calls: 0 outliers
Payment Delay: 0 outliers
Last Interaction: 0 outliers
Churn: 0 outliers
Total Spend: 0 outliers


### Normalization

In [18]:
Sub_Type = {
    'Standard': 1,
    'Basic': 2,
    'Premium': 3
}

df['Subscription Type ID'] = df['Subscription Type'].map(Sub_Type)

In [19]:
df[['Subscription Type ID','Subscription Type']]

,Subscription Type ID,Subscription Type
0,1,Standard
1,2,Basic
2,2,Basic
3,1,Standard
4,2,Basic
...,...,...
440828,3,Premium
440829,3,Premium
440830,1,Standard
440831,1,Standard


In [20]:
Contract_len = {
    'Monthly': 1,
    'Quarterly': 2,
    'Annual': 3
}

df['Contract Length ID'] = df['Contract Length'].map(Contract_len)

In [21]:
df[['Contract Length ID','Contract Length']]

,Contract Length ID,Contract Length
0,3,Annual
1,1,Monthly
2,2,Quarterly
3,1,Monthly
4,1,Monthly
...,...,...
440828,3,Annual
440829,3,Annual
440830,2,Quarterly
440831,2,Quarterly


In [22]:
Fact_Customer_Churn=df.loc[:,['CustomerID',                              # Table 1
                    'Tenure',
                    'Usage Frequency',
                    'Support Calls',
                    'Payment Delay',
                    'Total Spend',
                    'Last Interaction',
                    'Churn',
                    'Subscription Type ID',
                    'Contract Length ID']]

In [23]:
Fact_Customer_Churn

,CustomerID,Tenure,Usage Frequency,Support Calls,Payment Delay,Total Spend,Last Interaction,Churn,Subscription Type ID,Contract Length ID
0,2,39,14,5,18,932.00,17,1,1,3
1,3,49,1,10,8,557.00,6,1,2,1
2,4,14,4,6,18,185.00,3,1,2,2
3,5,38,21,7,7,396.00,29,1,1,1
4,6,32,20,5,8,617.00,20,1,2,1
...,...,...,...,...,...,...,...,...,...,...
440828,449995,54,15,1,3,716.38,8,0,3,3
440829,449996,8,13,1,20,745.38,2,0,3,3
440830,449997,35,27,1,5,977.31,9,0,1,2
440831,449998,55,14,2,0,602.55,2,0,1,2


In [24]:
Dim_Customer = df[['CustomerID', 'Age', 'Gender']].drop_duplicates()         # Table 2

In [25]:
Dim_Customer

,CustomerID,Age,Gender
0,2,30,Female
1,3,65,Female
2,4,55,Female
3,5,58,Male
4,6,23,Male
...,...,...,...
440828,449995,42,Male
440829,449996,25,Female
440830,449997,26,Male
440831,449998,28,Male


In [26]:
Dim_Subscription = df[['Subscription Type ID','Subscription Type']].drop_duplicates().reset_index(drop=True) # Table 3
# reset index --> because the previous indices was 0 1 5

In [27]:
Dim_Subscription

,Subscription Type ID,Subscription Type
0,1,Standard
1,2,Basic
2,3,Premium


In [28]:
Dim_Contract = df[['Contract Length ID','Contract Length']].drop_duplicates().sort_values('Contract Length ID').reset_index(drop=True)
# Table 4

In [29]:
Dim_Contract

,Contract Length ID,Contract Length
0,1,Monthly
1,2,Quarterly
2,3,Annual


In [30]:
Dim_Contract.dtypes

Contract Length ID     int64
Contract Length       object
dtype: object

In [32]:
# with pd.ExcelWriter('Cleaned_Customer_Churn_Dataset.xlsx', engine='xlsxwriter') as writer:
#     Fact_Customer_Churn.to_excel(writer, sheet_name='Fact_Customer_Churn', index=False)
#     Dim_Customer.to_excel(writer, sheet_name='Dim_Customer', index=False)
#     Dim_Subscription.to_excel(writer, sheet_name='Dim_Subscription', index=False)
#     Dim_Contract.to_excel(writer, sheet_name='Dim_Contract', index=False)